# <center>⚡️🏁 Welcome! Как запустить эти ваши ноутбуки с курса? 🤔📔 </center>

<img src='../images/choice_api.jpeg' align="right" width="400" height="400" >

## Оглавление ноутбука

 * [🚀 API от OpenAI.](part#1)
 * [🛠 API от OpenAI с __нашего сервера__.](#part2)
 * [🤐 Чат-модели и объект ChatOpenAI](#part3)
 * [🔧 Open Source LLM модели и API от HuggingFaceHub.](#part4)
 * [🔋 Локальный запуск LLM без интернета и API ключей.](#part5)
 * [🧸 Выводы и заключения.](#part6)

In [1]:
import os
import warnings
from getpass import getpass  # Для скрытого ввода API ключей
warnings.filterwarnings("ignore")

In [3]:
# Установим нужные библиотеки 
# !pip install openai langchain==0.0.312 huggingface_hub transformers -q

# <center id="part1">  🤖 Как использовать API от `OpenAI` напрямую?!

<div class="alert alert-info">
    
**🔑 Получение ключа для использования `OpenAI API`**

> При создании своего первого ключа, вы получите бесплатные 5$ для использования платформы.

0. Если вы из РФ, то используйте __VPN__ при заходе на сайт. 
1. Если нет аккаунта от OpenAI, то для регистрации потребуется __зарубежная симкарта__ (можно приобрести в интернете, либо купить доступ к уже зарегистрированному аккаунту, ~100 руб.
2. Перейдите по [ссылке](https://platform.openai.com/account/api-keys) и авторизуйтесь на сайте `OpenAI`.
3. После успешной авторизации на сайте, нажмите на кнопку `+ Create new secret key` и введите любое название для ключа.
4. Скопируйте полученный ключ и поместите его в переменную среды `OPENAI_API_KEY`
5. Для оплаты новых нокенов из РФ, потребуется __иностранная карта__!

* ⚙️ Начнем с прямых вызовов к API от OpenAI.
* 🗂 [Документация](https://platform.openai.com/docs/api-reference/making-requests) по использованию API
* 👨‍🍳 Полезный [Cookbook](https://cookbook.openai.com/)

In [2]:
# openai_api_key = "Введите ваш OpenAI API ключ"
openai_api_key = getpass(prompt="Введите ваш OpenAI API ключ")
os.environ["OPENAI_API_KEY"] = openai_api_key

Введите ваш OpenAI API ключ ········


Рассмотрим некоторые параметры функции `ChatCompletion.create()`

О всех параметрах можно узнать на [странице](https://platform.openai.com/docs/api-reference/chat/create)

In [4]:
import openai

openai.api_key = openai_api_key
openai.Engine.list() # Выведет все возможные модели от OpenAI, которые можно указать в параметре model

In [ ]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="api",
)
messages = [
    {
        "role": "user",  # Роль - ассистент или юзер
        "content": "1+1",  # Сам промпт для подачи в chatGPT
    }
]

chat_completion = client.chat.completions.create(
    max_tokens=8000,
    model="gpt-3.5-turbo",  # модель для выбора
    messages=messages,  # сообщение
    temperature=0,  # степень креативности ответа
)

print(chat_completion.choices[0].message.content)

<div class="alert alert-info">

**😎 Два обязательных параметра**
    
- `messages` - список объектов сообщений, где каждый объект имеет два обязательных поля:
    - `content` - содержимое вашего сообщения
    - `role` - роль автора сообщений (`system`, `user` или `assistant`)
- `model` - модель, используемая для генерации сообщения (доступные модели на [странице](https://platform.openai.com/docs/models/model-endpoint-compatibility))

<div class="alert alert-info">

**🥸 Полезно использовать**
    
- `max_tokens` - максимальное количество токенов, генерируемых в сообщении
- `n` - сколько вариантов ответа должно быть сгенерировано для каждого входного сообщения (по умолчанию 1)
- `temperature` - влияет на степень случайности генерации (принимает значения от 0 до 2, по умолчанию 1). Высокое значение температуры делает вероятности всех слов в выводе более равномерными, что приводит к более случайному и разнообразному тексту. При низком значении температуры модель будет склонна генерировать более предсказуемый и детерминированный текст.
- `top_p` - тоже влияет на степень случайности генерации. Определяет, какой процент наиболее вероятных слов будет включен в генерацию. Например, если top-p равно 0.8, то модель будет использовать вероятности слов так, чтобы включить в генерацию слова, составляющие 80% верхней части распределения вероятностей.

> Рекомендуем изменять либо параметр `temperature`, либо `top_p`, но не оба одновременно

```python
{
  "id": "chatcmpl-8B6YmCk5OiK7Z6RkRrkx72mEQaPGF",
  "object": "chat.completion",
  "created": 1697657516,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "1+1 equals 2."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 10,
    "completion_tokens": 7,
    "total_tokens": 17
  }
}
```

<div class="alert alert-info">

**🧐 А что получается в выводе?**

- `id` - уникальный идентификатор запроса
- `object` - тип возвращаемого объекта
- `created` - время запроса в timestamp-формате
- `model` - полное название модели, которая использовалась для генерации ответа
- `choices` - список объектов ответа (только один, если не изменяли параметр `n`)
    - `index` - индекс ответа в списке вариантов
    - `message` - объект сообщения, сгенерированный моделью, содержит `role` и `content`
    - `finish_reason` - причина, по которой модель закончила генерировать ответ
        - `stop` - достигнута естественная точка остановки
        - `length` - достигнуто максимальное количество токенов, указанное в запросе
        - `content_filter` - сгенерированный текст содержит нежелательный материал, который нарушает правила
- `usage` - количество используемых токенов
    - `prompt_tokens` - для распознавания вопроса
    - `completition_tokens` - для генерации ответа
    - `total_tokens` - для всего запроса (вопрос + ответ)

<div class="alert alert-info">
    
✅ __Все!__ Этого уже достаточно, чтобы просто получать ответы от `ChatGPT` кодом. Если для ваших задач этого хватает, то можно использовать уже так. Для удобства, можно еще обернуть дополнительно в функцию.

# <center id="part2"> 🤖 `API` от `СhatGPT` с ключом курса (рекомендуется)

In [ ]:
# Для работы в колабе, загрузите наш скрипт для использования ChatGPT с сервера курса!
# !wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

In [1]:
from utils import OpenAI

# course_api_key = 'Введите ваш API-ключ, который получили в боте'
# course_api_key = getpass(prompt="Введите ваш API-ключ, который получили в боте")

<div class="alert alert-info">

**🔑 Получение ключа для использования модели на нашем сервере через API**

1. Перейдите по [ссылке](https://t.me/llm_course_bot) в нашего телеграмм-бота.
2. Авторизуйтесь, указав в нем свой `Stepik id`, с которым занимаетесь на курсе.
3. Получите свой ключ и используйте наш API для прохождения курса.
4. Также в боте можно проверить действие вашего ключа, узнать сколько осталось доступных токенов и дату, когда ключ перестанет действовать.

> Если возникли трудности, ошибки, вопросы или бот не работает, то смело пишите в чат курса, мы вам поможем.

In [3]:
from config import course_api_key, openai_api_key
from utils import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    course_api_key=course_api_key,  # ключ для доступа к апи
)

In [3]:
prompt = "1+1"

messages = [{
    "role": "user",  # Роль - ассистент или юзер
    "content": prompt ,  # Сам промпт для подачи в chatGPT
    }]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",  # модель для выбора
    messages=messages,  # сообщение
    temperature=0,  # степень креативности ответа
)

print(response.choices[0]["message"])

{'role': 'assistant', 'content': '1+1 equals 2.'}

<div class="alert alert-info">

При выполнении функции при помощи `print(response)`, кроме сгенерированного ответа, на экран выводится дополнительная информация:
- `prompt_tokens` - сколько токенов ушло на распознование вашего `prompt`
- `completion_tokens` - сколько токенов ушло на генерацию ответа
- `available_tokens` - сколько у вас осталось доступных токенов (можно также проверить в боте)

# <center id="part3"> 🤐 Чат модели и объект `ChatOpenAI` от `LangChain`

In [6]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate  # <-- Класс от langchain!

template = """Вопрос: {question}
Ответ: Дай короткий ответ"""

prompt = PromptTemplate(template=template, input_variables=["question"])

openai_llm = ChatOpenAI(temperature=0.0, openai_api_key=openai_api_key)

llm_chain = LLMChain(prompt=prompt, llm=openai_llm)

question = "Когда человек первый раз полетел в космос?"

print(llm_chain.run(question))

PermissionDeniedError: <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Attention Required! | Cloudflare</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/cf.errors.css" />
<!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->
<style>body{margin:0;padding:0}</style>


<!--[if gte IE 10]><!-->
<script>
  if (!navigator.cookieEnabled) {
    window.addEventListene

# <center id="part4"> 🤘 Чат модели и наша версия `ChatOpenAI` (рекомендуется)

In [3]:
from utils import ChatOpenAI  # <-- наш класс!

template = """Вопрос: {question}
Ответ: Дай короткий ответ
"""

prompt = PromptTemplate(template=template, input_variables=["question"])

openai_llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

llm_chain = LLMChain(prompt=prompt, llm=openai_llm)

question = "Когда человек первый раз полетел в космос?"

print(llm_chain.run(question))

12 апреля 1961 года


# <center > 🤗 Исследуем мир Open Source: `API` от `HuggingFaceHub`

In [13]:
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'Введите ваш HuggingFaceHub API ключ'
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass(prompt="Введите ваш HuggingFaceHub API ключ")

Введите ваш HuggingFaceHub API ключ ········


<div class="alert alert-info">
    
**Получение ключа для использования `HuggingFaceHub` `API`**

1. Авторизуйтесь или зарегистрируйтесь на сайте [HuggingFace](https://huggingface.co/).
2. Перейдите по [ссылке](https://huggingface.co/settings/tokens) и нажмите на кнопку `New token`.
3. Введите любое название и нажмите на кнопку `Generate a token`.
4. Скопируйте полученный ключ и поместите его в переменную среды `HUGGINGFACEHUB_API_TOKEN`


In [14]:
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

# initialize HF LLM
hf_llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
)

# build prompt template for simple question-answering
template = """Question: {question}.

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=hf_llm)

question = "When did man first fly into space?"

print(llm_chain.run(question))

 The first manned space flight was on April 12, 1961. The


<div class="alert alert-info">
    
✅ __Все!__ теперь мы можете даже не спускать свои деньги не токены! Вот вам весь зоопарк моделей на [HuggingFace.com](https://huggingface.co/models?pipeline_tag=text-generation&sort=trending). Выбирайте любую хайповую модельку, хоть даже [meta-llama/Llama-2-7b](https://huggingface.co/meta-llama/Llama-2-7b) и кайфуйте! Нужен только токен. 

# <center id="part5"> 💻 Ваш собственный ChatGPT: запуск LLM локально

> Для локального использования моделей `HuggingFace` (на вашем локальном компьютере или сервере), необходима библиотека `transformers`, а также должны быть скачаны веса необходимой модели.

In [16]:
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

bloom = HuggingFacePipeline.from_model_id(
    model_id="bigscience/bloom-1b7",  # Название и версия модели, позже покажем вам весь LLM-зоопарк
    task="text-generation",  # Вид задачи, в нашем случае - генерация текста
    model_kwargs={
        "temperature": 0,
        "max_length": 64,
    },  # все те же креативность и максимальная длина
    device=0,  # Номер GPU карточки, если есть!
)

2023-10-22 15:00:29.954165: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-22 15:00:30.578701: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[2023-10-22 15:00:31,939] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [17]:
template = """Question: {question}.

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=bloom)

question = "When did man first fly into space?"

print(llm_chain.run(question))

 The first man to fly into space was the American astronaut
John Glenn, who was born in New York City on January 1, 1903. He was
the son of a carpenter and a housewife. He was a very intelligent boy,
and


<div class="alert alert-info">

✅ __Все!__ Теперь вам даже интерент не нужен. Грузите себе локально и ускоряйте! Однако, следите за качеством выбираемой модели. Как вы убедились выше, не у всех моделей качество генерации высокое!

# <center id="part6"> 🧸 Выводы и заключения ✅
    
* ✅ Вы умеете запускать код через API
* ✅ Вы знаете откуда стягивать модели
* 🔑 Даже если у вас нет VPN, иностранного номера и счета, то у вас есть ключ от команды курса!
* ✅ Под каждую задачу есть своя оптимальная модель подходящего качества и размера, не одним ChatGPT едины!